# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [9]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [10]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [ ]:
# TODO: Create a .env file with the following variables - DONE
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [3]:
# TODO: Load environment variables - DONE
load_dotenv()

True

### VectorDB Instance

In [4]:
# TODO: Instantiate your ChromaDB Client - DONE
# Choose any path you want
chroma_client = chromadb.PersistentClient(path="./chroma_db")

### Collection

In [5]:
# TODO: Pick one embedding function - DONE
# If picking something different than openai, 
# make sure you use the same when loading it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
# TODO: Create a collection - DONE
# Choose any name you want
try:
    chroma_client.delete_collection("udaplay")
except:
    pass

collection = chroma_client.create_collection(
    name="udaplay",
    embedding_function=embedding_fn
)

### Add documents

In [7]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [8]:
# TODO: Query the collection to demonstrate semantic search
print("--- Semantic Search Demo ---")
results = collection.query(
    query_texts=["Find me a game about racing"],
    n_results=2
)
print(json.dumps(results, indent=2))

--- Semantic Search Demo ---
{
  "ids": [
    [
      "001",
      "003"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
      "[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics."
    ]
  ],
  "uris": null,
  "included": [
    "metadatas",
    "documents",
    "distances"
  ],
  "data": null,
  "metadatas": [
    [
      {
        "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
        "Name": "Gran Turismo",
        "Genre": "Racing",
        "Platform": "PlayStation 1",
        "YearOfRelease": 1997,
        "Publisher": "Sony Computer Entertainment"
      },
      {
        "Description": "A comprehensive racing simulator featuring a vast 